In [ ]:
from llama_index.core import (
    Document,
    Settings,
    SimpleDirectoryReader,
    VectorStoreIndex
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
import pandas as pd
from trulens_eval import Tru, TruLlama
from utils import (
    build_automerging_index,
    build_sentence_window_index,
    evaluate_engine,
    get_automerging_query_engine,
    get_feedback_func,
    get_sentence_window_query_engine,
    setup 
)
import warnings

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option("display.max_colwidth", None)
setup() 

In [ ]:
!mkdir -p '../data'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O '../data/paul_graham_essay.txt'

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["../data/paul_graham_essay.txt"]
).load_data()

In [ ]:
len(documents), type(documents)

In [ ]:
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What is Paul Graham known for?")
str(response)

In [ ]:
tru = Tru()
tru.reset_database()

In [ ]:
with open('eval_questions.txt', 'r') as file:
    eval_questions = file.read().split("\n")

In [ ]:
feedbacks = get_feedback_func()

In [ ]:
evaluate_engine(
    query_engine=query_engine,
    feedbacks=feedbacks,
    app_id="Direct Query Engine",
    eval_questions=eval_questions,
    verbose=False
)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [ ]:
records[["input", "output"] + feedback]

In [ ]:
window_index = build_sentence_window_index(documents, Settings.llm)
window_engine = get_sentence_window_query_engine(window_index)

In [ ]:
evaluate_engine(
    query_engine=window_engine,
    feedbacks=feedbacks,
    app_id="Window Engine",
    eval_questions=eval_questions,
    verbose=False
)

In [ ]:
merging_index = build_automerging_index(documents, Settings.llm)
merging_engine = get_automerging_query_engine(merging_index)

In [ ]:
evaluate_engine(
    query_engine=merging_engine,
    feedbacks=feedbacks,
    app_id="Merging Engine",
    eval_questions=eval_questions,
    verbose=False
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

tru.get_leaderboard(app_ids=[])
tru.run_dashboard()